In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
#botdata = pd.read_csv('botAccounts.csv')
#gooddata= pd.read_csv('GoodAccounts.csv', encoding = 'ISO-8859-1')

In [2]:
testbot = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/bots_data.csv", encoding = 'ISO-8859-1')
testgood = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/nonbots_data.csv", encoding = 'ISO-8859-1')

In [3]:
df = testbot.append(testgood)

In [146]:
testbot.shape

(1056, 20)

In [147]:
testgood.shape

(1176, 20)

In [143]:
df.shape

(2232, 20)

In [46]:
#modify the column name to fit
def adjust_column_name(df):
    for x in df.columns.values:
        df = df.rename(columns={x: x.lower()})
    df = df.rename(columns={'listed_count' : 'listedcount'})

In [6]:
# split the dataset to training & testing
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [190]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

In [ ]:
df = testbot.append(testgood).fillna("")

In [191]:
def nb_dt(nb_column_name):
    train, test = train_test_split(df, test_size = 0.2)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','nb_result']
    count_vect = CountVectorizer()
    
    def convert(column):
        X_word_counts = count_vect.fit_transform(column)
        count_vect.vocabulary_.get(u'algorithm')
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    def convert2(column):
        X_word_counts = count_vect.transform(test[nb_column_name])
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    #Train
    clf = tree.DecisionTreeClassifier(random_state=0)
    M_clf = MultinomialNB().fit(convert(train[nb_column_name]), train['bot'])
    train['nb_result'] = M_clf.predict(convert(train[nb_column_name]))
    clf.fit(train[numfeatures],train["bot"])
    #Test
    test['nb_result'] = M_clf.predict(convert2(train[nb_column_name]))
    
    #Evaluate
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [precison, recall, f1, roc]

In [195]:
for _ in range(20):
    print(nb_dt('description'))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.89655172413793105, 0.78787878787878785, 0.83870967741935487, 0.84532828282828276]
[0.86538461538461542, 0.8035714285714286, 0.83333333333333337, 0.83900544522741827]
[0.85572139303482586, 0.85572139303482586, 0.85572139303482586, 0.86891760708651866]
[0.86190476190476195, 0.84976525821596249, 0.85579196217494102, 0.86291681714216928]
[0.85116279069767442, 0.83561643835616439, 0.84331797235023032, 0.84763278058159097]
[0.88584474885844744, 0.83620689655172409, 0.86031042128603097, 0.85996391339214118]
[0.83177570093457942, 0.839622641509434, 0.83568075117370888, 0.84321557607386588]
[0.84541062801932365, 0.84951456310679607, 0.84745762711864392, 0.85836724005962206]
[0.80660377358490565, 0.82608695652173914, 0.81622911694510736, 0.82762681159420293]
[0.84158415841584155, 0.80952380952380953, 0.8252427184466018, 0.83725135623869795]
[0.85784313725490191, 0.83732057416267947, 0.84745762711864414, 0.85773591733344057]
[0.82741116751269039, 0.7990196078431373, 0.81296758104738165, 0.8295

In [158]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [193]:
def dt():
    df = testbot.append(testgood)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count']
    train, test = train_test_split(df, test_size = 0.2)
    clf.fit(train[numfeatures],train["bot"])
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [precison, recall, f1, roc]

In [194]:
for _ in range(20):
    print(dt())

[0.81777777777777783, 0.84792626728110598, 0.83257918552036203, 0.8348326988579442]
[0.87142857142857144, 0.84331797235023043, 0.85714285714285721, 0.86296333400120218]
[0.82648401826484019, 0.85781990521327012, 0.84186046511627899, 0.84840147803036392]
[0.84651162790697676, 0.87081339712918659, 0.85849056603773588, 0.8660789674721564]
[0.865979381443299, 0.79620853080568721, 0.82962962962962972, 0.84301951964013178]
[0.80487804878048785, 0.82914572864321612, 0.81683168316831678, 0.83392770303128549]
[0.8133971291866029, 0.80568720379146919, 0.80952380952380953, 0.82021648325166685]
[0.85416666666666663, 0.75229357798165142, 0.80000000000000004, 0.81501141781178643]
[0.82857142857142863, 0.79090909090909089, 0.80930232558139537, 0.81615939126952342]
[0.86206896551724133, 0.80645161290322576, 0.83333333333333337, 0.84235624123422159]
[0.80769230769230771, 0.89839572192513373, 0.85063291139240516, 0.87227478403948999]
[0.83091787439613529, 0.80000000000000004, 0.81516587677725139, 0.8245